<a href="https://colab.research.google.com/github/wangyeye66/projects/blob/main/NLP_seq_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]

In [ ]:
from transformers import pipeline

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# example
tweet = "I love this!"
result = sentiment_pipeline(tweet)
result

[{'label': 'POSITIVE', 'score': 0.9998764991760254}]

### fine tune pretrained model


In [ ]:
### training model from scratch
import torch
import pandas as pd

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load dataset
data_columns = ["target", "ids", "date", "flag", "user", "text"]
data_encoding = "ISO-8859-1"
file_path = '/content/drive/MyDrive/colab data/twitter_sentiment.csv'

df_total = pd.DataFrame()

# Read the file in chunks
chunksize = 10000  # Adjust based on your needs and memory constraints
try:
    for chunk in pd.read_csv(file_path, encoding=data_encoding, names=data_columns, header=None, chunksize=chunksize, on_bad_lines='skip'):
        df_total = pd.concat([df_total, chunk], ignore_index=True)
except pd.errors.ParserError as e:
    print(f"Skipping a problematic chunk: {e}")

# Check the DataFrame
df_total.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Skipping a problematic chunk: Error tokenizing data. C error: EOF inside string starting at row 2042802


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df = df_total[['target', 'text']]
df.dropna(subset=['text'], inplace=True)
df = df.rename(columns={'target': 'labels'})

print(df.isnull().any())

# randomly select 20000 pieces of data
df = df.sample(n=20000)
# 1 positive, 0 negative
df['labels'] = df['labels'].astype(int).replace(4, 1)
df['labels'].value_counts()

<ipython-input-7-e9dcb5a245cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['text'], inplace=True)


labels    False
text      False
dtype: bool


0    14064
1     5936
Name: labels, dtype: int64

In [ ]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# Split the data into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2)

# Convert the train and validation DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function to preprocess the text
def tokenize_function(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)
    # Include the labels
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize the train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# send model to gpu
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,  # Evaluation and logging each 50 steps
    save_steps=100,  # Save checkpoint every 100 steps
    load_best_model_at_end=True,
)

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
50,0.645500,0.604123
100,0.595400,0.537779
150,0.459500,0.417027
200,0.456900,0.453018
250,0.453000,0.414813
300,0.472700,0.390426
350,0.390700,0.446058
400,0.387800,0.375734
450,0.470700,0.377754
500,0.341600,0.402906


TrainOutput(global_step=10000, training_loss=0.11043577696009306, metrics={'train_runtime': 3396.9002, 'train_samples_per_second': 47.102, 'train_steps_per_second': 2.944, 'total_flos': 1.05244422144e+16, 'train_loss': 0.11043577696009306, 'epoch': 10.0})

In [ ]:
# Save the model
model.save_pretrained("./sentiment_model")

### evaluate

In [ ]:
# Make prediction on new tweets
from transformers import pipeline
import torch

In [ ]:
# Load the trained model and tokenizer using the pipeline
model_path = "./sentiment_model"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_path, tokenizer='bert-base-uncased', framework='pt', device=0 if torch.cuda.is_available() else -1)

In [ ]:
# New tweets for prediction
new_tweets = ["I'm thrilled with the new update!", "This is the worst thing ever!"]

In [ ]:
# Perform sentiment analysis
predictions = sentiment_pipeline(new_tweets)

In [ ]:
predictions

[{'label': 'LABEL_1', 'score': 0.9738933444023132},
 {'label': 'LABEL_0', 'score': 0.9702444076538086}]

In [ ]:
### Pre-trained model for seq classification
# BERT (Base and Large) for Sequence Classification
bert_base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_base_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bert_large_model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
bert_large_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# DistilBERT for Sequence Classification
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# RoBERTa for Sequence Classification
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# ALBERT for Sequence Classification
albert_model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')
albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')